In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
paths = [
    "files/apostila.pdf",
    "files/LLM.pdf",
]

pages = []

## Cria uma lista com todos os documentos
for path in paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    pages.extend(docs)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50, separators=["\n\n", "\n", ". ", " ", ""]
)

docs = splitter.split_documents(pages)

normalized_docs = []

for doc in docs:
    # Remove repeated dots, dashes and spaces
    cleaned_text = doc.page_content
    cleaned_text = " ".join(cleaned_text.split())  # Remove extra whitespace
    cleaned_text = cleaned_text.replace("....", "")
    cleaned_text = cleaned_text.replace("...", "")
    cleaned_text = cleaned_text.replace("..", "")
    cleaned_text = cleaned_text.replace("----", "-")
    cleaned_text = cleaned_text.replace("---", "-")
    cleaned_text = cleaned_text.replace("--", "-")
    cleaned_text = cleaned_text.replace("  ", "")
    cleaned_text = cleaned_text.strip()

    # Create new document with cleaned text
    doc.page_content = cleaned_text
    normalized_docs.append(doc)

print(len(docs))

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

146


In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## Banco vetorial

from langchain_chroma import Chroma

dir = "db/chroma_db_vectorstore"

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=dir,
)

In [15]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)



In [16]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_type="mmr")
)

In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Template melhorado
question_prompt_template = PromptTemplate.from_template(
    """
Você é um especialista em Inteligência Artificial e Modelos de Linguagem.

Baseado apenas nas informações abaixo dos documentos, responda à pergunta de forma completa, precisa e didática. Não invente informações — responda apenas com base no que foi encontrado.

Documentos:
{context_str}

Pergunta:
{question}

Resposta:
"""
)

refine_prompt_template = PromptTemplate.from_template(
    """
Você já escreveu uma resposta inicial baseada em alguns documentos.

Aqui está a resposta inicial:
{existing_answer}

Baseado no novo documento abaixo, você pode melhorar e expandir a resposta anterior, se necessário. Seja preciso e mantenha a didática.

Novo Documento:
{context_str}

Se o novo documento não trouxer informações relevantes, mantenha a resposta original.

Resposta Refinada:
"""
)


# Agora passa esse template pra RetrievalQA
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(search_type="mmr"),
    chain_type="refine",
    chain_type_kwargs={
        "question_prompt": question_prompt_template,
        "refine_prompt": refine_prompt_template,
    },
)


ask = "O que é Hugging Face?"

response = chat_chain.invoke({"query": ask})

print(response["result"])

Hugging Face é uma comunidade de código aberto que reúne uma ampla variedade de modelos de Inteligência Artificial contribuídos por centenas de milhares de desenvolvedores. Esses modelos podem ser utilizados para uma variedade de casos de uso, como geração de texto, resumo e classificação. Embora a comunidade esteja rapidamente alcançando o desempenho de modelos proprietários, como o GPT-4, ainda há desafios a serem superados para igualar o desempenho dos modelos mais avançados. No entanto, a colaboração e a diversidade de modelos disponíveis na Hugging Face tornam-na uma ferramenta poderosa para desenvolvedores e pesquisadores que buscam soluções de Inteligência Artificial inovadoras e eficazes.

Além disso, é importante ressaltar que o aprendizado em Python é fundamental para quem deseja se aprofundar em Inteligência Artificial. O Python oferece uma ampla gama de recursos, como orientação a objetos, programação funcional, metaprogramação, interface gráfica, expressões regulares, thre